In [144]:
import plotly.graph_objects as go
import ipywidgets as widgets
import pandas as pd
import numpy as np

df = pd.read_csv('TaskSkill.csv')
# Combine Skill group and Skill component into a single categorical variable
df['Combined'] = df['Skill group'] + ' - ' + df['Skill component']
# Sort the DataFrame by the 'Combined' column
df_sorted = df.sort_values(by='Combined', ascending=False)

# Build parcats dimensions
categorical_dimensions = ['Skill group', 'Skill component', 'Task']

dimensions = [dict(values=df_sorted[label], label=label) for label in categorical_dimensions]

# Build colorscale
color = np.zeros(len(df), dtype='uint8')
colorscale = [[0, 'gray'], [0.15, 'gray'],
              [0.15, 'firebrick'], [0.3, 'firebrick'],
              [0.3, 'blue'], [0.45, 'blue'],
              [0.45, 'green'], [0.6, 'green'],
              [0.6, 'yellow'], [0.75, 'yellow'],
              [0.75, 'purple'], [0.9, 'purple'],
              [0.9, 'orange'], [1, 'orange']
]
cmin = 0
cmax = 7

# Create a new DataFrame with unique Skill group labels and their corresponding Combined values
unique_groups = df.groupby('Skill group')['Combined'].max().reset_index()

# Create the Scatter plot for unique Skill group labels
scatter_data = go.Scatter(x=df_sorted['Domain'], y=df_sorted['Combined'],
                          marker={'color': color, 'cmin': cmin, 'cmax': cmax,
                                  'colorscale': colorscale, 'showscale': False,
                                  #'colorbar': {'tickvals': [0, 1, 2], 'ticktext': ['None', 'Red', 'Blue']}
                                 },
                          hoverinfo='none',  # Disable hover information for the scatter plot
                          mode='markers')

# Create the Scatter plot for individual Skill components
parcats_data = go.Parcats(
    domain={'y': [0, 0.4]}, dimensions=dimensions,
    line={'colorscale': colorscale, 'cmin': cmin,
          'cmax': cmax, 'color': color, 'shape': 'hspline'},
    hoverinfo='none',  # Disable hover information for the parcats plot
    bundlecolors=True  # Group Skill component labels by Skill group
)

# Build figure as FigureWidget
fig = go.FigureWidget(
    data=[scatter_data, parcats_data]
)

fig.update_layout(height=1600,
                   xaxis={
                        'title': 'Domain',
                        'categoryorder': 'array',
                        'categoryarray': ['Innovation', 'Entrepreneurship', 'Commercialisation']
                    },
                  yaxis={'title': 'Skills', 'domain': [0.6, 1]},
                  dragmode='lasso', hovermode='closest')

# Build color selection widget
color_toggle = widgets.ToggleButtons(
    options=['None', 'Red', 'Blue', 'Green', 'Yellow', 'Purple', 'Orange'],  # Additional color choices
    index=1, description='Brush Color:', disabled=False
)
# Update color callback
def update_color(trace, points, state):
    # Define color mappings
    color_mapping = {'None': 0, 'Red': 2.5, 'Blue': 3.5, 'Green': 4.5, 'Yellow': 5.5, 'Purple': 6.5, 'Orange': 7.5}

    # Compute new color array
    new_color = np.array(fig.data[0].marker.color)
    selected_color = color_toggle.value
    new_color[points.point_inds] = color_mapping[selected_color]

    # Adjust line width only if color is not 'None'
    line_width = 1
    if selected_color != 'None':  # If color is not 'None'
        line_width = 50

    with fig.batch_update():
        # Update scatter color
        fig.data[0].marker.color = new_color

        # Update parcats colors
        fig.data[1].line.color = new_color
        fig.data[1].line.width = line_width  # Update line width
        
# Register callback on scatter selection...
fig.data[0].on_selection(update_color)
# and parcats click
fig.data[1].on_click(update_color)

# Display figure
widgets.VBox([color_toggle, fig])
